# Advent of Code 2024

Let's try out ocaml this year!  I read a book about Standard ML like 20 years ago, but remember very little.  So there will be a learning curve, but it shouldn't be as bad as APL.

Is this thing on?

In [1]:
2+2;;

- : int = 4


## Reusable utilities

This section is for stuff I get tired of copying and pasting.

### Ok, what is "core"?

On Day 3, while looking for basic I/O utilities again (is there really no "read entire file" library?), I learned there's an entire alternative standard library written by Jane Street.  This feels weird, but whatever.

In [2]:
#use "topfind" ;;

- : unit = ()
Findlib has been successfully loaded. Additional directives:
  #require "package";;      to load a package
  #list;;                   to list the available packages
  #camlp4o;;                to load camlp4 (standard syntax)
  #camlp4r;;                to load camlp4 (revised syntax)
  #predicates "p,q,...";;   to set these predicates
  Topfind.reset();;         to force that packages will be reloaded
  #thread;;                 to enable threads

- : unit = ()


In [3]:
#require "core" ;;

/Users/jered/.opam/4.12.0/lib/base/base_internalhash_types: added to search path
/Users/jered/.opam/4.12.0/lib/base/base_internalhash_types/base_internalhash_types.cma: loaded
/Users/jered/.opam/4.12.0/lib/base/caml: added to search path
/Users/jered/.opam/4.12.0/lib/base/caml/caml.cma: loaded
/Users/jered/.opam/4.12.0/lib/base/shadow_stdlib: added to search path
/Users/jered/.opam/4.12.0/lib/base/shadow_stdlib/shadow_stdlib.cma: loaded
/Users/jered/.opam/4.12.0/lib/sexplib0: added to search path
/Users/jered/.opam/4.12.0/lib/sexplib0/sexplib0.cma: loaded
/Users/jered/.opam/4.12.0/lib/base: added to search path
/Users/jered/.opam/4.12.0/lib/base/base.cma: loaded
/Users/jered/.opam/4.12.0/lib/fieldslib: added to search path
/Users/jered/.opam/4.12.0/lib/fieldslib/fieldslib.cma: loaded
/Users/jered/.opam/4.12.0/lib/ppx_compare/runtime-lib: added to search path
/Users/jered/.opam/4.12.0/lib/ppx_compare/runtime-lib/ppx_compare_lib.cma: loaded
/Users/jered/.opam/4.12.0/lib/ppx_enumerate/run

In [4]:
open Core

### Oh God, what have I done

Apparently this is an overlay library, and just blows away much of the core language.  Now my `List.map` function is backwards!  That's... incredibly antisocial, actually.  I guess what did I expect from a quantitative finance company.

In [5]:
List.map ((+) 0) [1;2;3]

File "[5]", line 1, characters 9-16:
1 | List.map ((+) 0) [1;2;3]
             ^^^^^^^
Warning 5 [ignored-partial-application]: this function application is partial,
maybe some arguments are missing.


error: compile_error

In [6]:
List.map [1;2;3] ((+) 0)

- : int list = [1; 2; 3]


Hang on, let me just rewrite everything now.

In [7]:
open Core.List

Apparently I need this too.  Have I mentioned it's impossible to Google search for punctuation?  Ugh.

### Reading stuff from files

We seem to have to read stuff from files a lot.

In [8]:
(* Core has a way to do this *)
In_channel.read_lines "day1_example.txt"

- : Base.string Base.list =
["3   4"; "4   3"; "2   5"; "1   3"; "3   9"; "3   3"]


Now there are different combinators, too.  How do these work?

In [9]:
[1;2;3] >>| ((+) 1)

- : int list = [2; 3; 4]


In [10]:
(*let read_numbers_from_file file_name =
  let file = open_in file_name in
  let output = ref [] in
  try
    while true do
      let line = input_line file
               |> Str.split (Str.regexp " +")
               |> List.map int_of_string in
      output := line :: !output
    done;
    assert false
  with
    End_of_file -> List.rev !output *)

let read_numbers_from_file file_name =
  let parse line =
    line |> Str.split (Str.regexp " +") >>| int_of_string in
  In_channel.read_lines file_name >>| parse

val read_numbers_from_file : Base.string -> int list list = <fun>


### This keeps coming up

In [11]:
let pair_of_list xs =
  match xs with
  | [x; y] -> (x, y)
  | _ -> failwith "expecting pair"

val pair_of_list : 'a list -> 'a * 'a = <fun>


## Day 1

[Link](https://adventofcode.com/2024/day/1)

### Part 1

They want me to read two columns from a file, sort them, and sum the differences.

In [12]:
let read_pairs_from_file file_name =
  read_numbers_from_file file_name
  >>| pair_of_list

val read_pairs_from_file : Base.string -> (int * int) list = <fun>


In [13]:
read_pairs_from_file "day1_example.txt"

- : (int * int) list = [(3, 4); (4, 3); (2, 5); (1, 3); (3, 9); (3, 3)]


Ok, that was a little ugly, but it could be worse.  Let's use some of this great type matching stuff.

In [14]:
let rec unzip pairs =
  match pairs with
  | [] -> ([], [])
  | (x, y) :: rest ->
    let (xs, ys) = unzip rest in
      (x::xs, y::ys)

val unzip : ('a * 'b) list -> 'a list * 'b list = <fun>


In [15]:
unzip([(1,2); (3,4); (5,6)])

- : int list * int list = ([1; 3; 5], [2; 4; 6])


That was fun.  ~But there's a library for this.~  Update: Ok, core seems to actually call this `zip` and `unzip`...

In [16]:
List.unzip [(1,2); (3,4); (5,6)]

- : int list * int list = ([1; 3; 5], [2; 4; 6])


In [17]:
zip [1;3;5] [2;4;6]

- : (int * int) list Core.List.Or_unequal_lengths.t =
Core.List.Or_unequal_lengths.Ok [(1, 2); (3, 4); (5, 6)]


Oh no, I overwrote `compare` when I did `open Core.List` to try to get the list combinators.  What a mess.

In [18]:
compare

- : ('a -> 'a -> int) -> 'a list -> 'a list -> int = <fun>


In [19]:
Stdlib.compare

- : 'a -> 'a -> int = <fun>


Hmm, post `zip`ping, my list type is infected with `Core.List.Or_unequal_lengths.t` and I can't do unsanctioned list stuff with it.  I guess I have to use `zip_exn` to fail hard.

In [20]:
let day1_part1 pairs =
  let (xs, ys) = unzip pairs in
  (zip_exn (sort xs Stdlib.compare) (sort ys Stdlib.compare)
    >>| fun (x, y) -> abs(x - y))
  |> List.fold ~f:(+) ~init:0

val day1_part1 : (int * int) list -> int = <fun>


~That is actually kind of pretty, although the syntax is meh.~  Let's be real, this is an ugly way to write a simple program.

In [21]:
read_pairs_from_file "day1_example.txt" |> day1_part1

- : int = 11


In [22]:
read_pairs_from_file "day1_input.txt" |> day1_part1

- : int = 1666427


### Part 2

Now we've got to count stuff.  There is no need to be especially clever about this.

In [23]:
let day1_part2 pairs =
  let (xs, ys) = unzip pairs in
  let count x = List.count ys ((=) x) in
    xs
    >>| (fun x -> x * (count x))
    |> List.fold ~f:(+) ~init:0

val day1_part2 : (Core_kernel.Int.t * Core_kernel.Int.t) list -> int = <fun>


In [24]:
read_pairs_from_file "day1_input.txt" |> day1_part2

- : int = 24316233


## Day 2

[Link](https://adventofcode.com/2024/day/2)

### Part 1

Today we have to take adjacent differences of things.

In [25]:
let rec differences list =
  match list with
  | [] -> []
  | [x] -> [] 
  | x :: y :: rest -> x - y :: differences (y :: rest)

val differences : int list -> int list = <fun>


In [26]:
differences [7; 6; 4; 2; 1]

- : int list = [1; 2; 2; 1]


I kind of wish this were point free?  Like I wanted to write `increasing || decreasing`.  Oh well.

In [27]:
let safe levels =
  let decreasing = List.for_all ~f:(fun(x) -> x < 0) in
  let increasing = List.for_all ~f:(fun(x) -> x > 0) in
  let in_range = List.for_all ~f:(fun(x) -> abs(x) >= 1 && abs(x) <= 3) in
  let check xs = (increasing xs || decreasing xs) && in_range xs in
    check (differences levels)

let count safety_check data =
  data
  |> List.filter ~f:safety_check
  |> List.length

val safe : int list -> bool = <fun>


val count : ('a -> bool) -> 'a list -> int = <fun>


In [28]:
read_numbers_from_file "day2_example.txt" |> count safe

- : int = 2


In [29]:
read_numbers_from_file "day2_input.txt" |> count safe

- : int = 442


### Part 2

Now we have to try fudging the data, I guess.

In [30]:
let rec ablate xs =
  match xs with
  | [] -> []
  | x :: rest -> rest :: (List.map ~f:(fun(xs) -> x :: xs)) (ablate rest)

val ablate : 'a list -> 'a list list = <fun>


In [31]:
ablate [1;2;3;4]

- : int list list = [[2; 3; 4]; [1; 3; 4]; [1; 2; 4]; [1; 2; 3]]


In [32]:
let safeish levels =
  safe levels ||
  (ablate levels |> List.exists ~f:safe)

val safeish : int list -> bool = <fun>


In [33]:
read_numbers_from_file "day2_example.txt" |> count safeish

- : int = 4


In [34]:
read_numbers_from_file "day2_input.txt" |> count safeish

- : int = 493


## Day 3

### Part 1

Today we are parsing instructions from strings full of junk.  There is a builtin regex library from, like, the 90s.  Since we're already doing this core stuff, I guess we may as well just use re2 and unlock the _sheer power_ of NFAs.  Stand back.

In [35]:
#require "re2" ;;

/Users/jered/.opam/4.12.0/lib/core_kernel/rope: added to search path
/Users/jered/.opam/4.12.0/lib/core_kernel/rope/rope.cma: loaded
/Users/jered/.opam/4.12.0/lib/re2/c: added to search path
/Users/jered/.opam/4.12.0/lib/re2/c/re2_c.cma: loaded
/Users/jered/.opam/4.12.0/lib/re2: added to search path
/Users/jered/.opam/4.12.0/lib/re2/re2.cma: loaded


In [36]:
let find_day3_instructions s =
  let mul = Re2.create_exn {|mul\((\d+,\d+)\)|} in
  Re2.find_all_exn ~sub:(`Index 1) mul s
  >>| fun (xs) -> (String.split ~on:',' xs
                   >>| int_of_string
                   |> pair_of_list)

val find_day3_instructions : string -> (int * int) list = <fun>


In [37]:
find_day3_instructions "xmul(2,4)%&mul[3,7]!@^do_not_mul(5,5)+mul(32,64]then(mul(11,8)mul(8,5))"

- : (int * int) list = [(2, 4); (5, 5); (11, 8); (8, 5)]


In [50]:
let day3_part1 s =
  find_day3_instructions s
  >>| (fun ((a, b)) -> a * b)
   |> fold ~init:0 ~f:(+)

val day3_part1 : string -> int = <fun>


In [39]:
In_channel.read_all "day3_example.txt" |> day3_part1

- : int = 161


In [40]:
In_channel.read_all "day3_input.txt" |> day3_part1

- : int = 170778545


### Part 2

Oh good, now we get to parse more instructions - we can turn multiplying on and off, how convenient.  Let's use a type thing.

In [41]:
type day3_instruction =
  Day3Mul of int * int
  | Day3Do
  | Day3Dont

type day3_instruction = Day3Mul of int * int | Day3Do | Day3Dont


In [42]:
let parse_day3_program s =
  let opcode = Re2.create_exn {|(don't\(\)|do\(\)|mul\(\d+,\d+\))|} in
  let parse_opcode args =
    match args with
    | "mul" :: x :: y :: rest -> Day3Mul (int_of_string x, int_of_string y)
    | "do" :: rest -> Day3Do
    | "don't" :: rest -> Day3Dont
    | _ -> failwith "unknown instruction" in
  Re2.find_all_exn ~sub:(`Index 1) opcode s
  >>| String.split_on_chars ~on:[','; '('; ')']
  >>| parse_opcode

val parse_day3_program : string -> day3_instruction list = <fun>


In [43]:
parse_day3_program "xmul(2,4)&mul[3,7]!^don't()_mul(5,5)+mul(32,64](mul(11,8)undo()?mul(8,5))"

- : day3_instruction list =
[Day3Mul (2, 4); Day3Dont; Day3Mul (5, 5); Day3Mul (11, 8); Day3Do;
 Day3Mul (8, 5)]


Let's shake things up and evaluate this imperatively.  Mwahaha, purists, take that.

In [44]:
let eval_day3_program s =
  let mul_enabled = ref true in
  let eval instruction =
    match instruction with
    | Day3Do -> mul_enabled := true; 0
    | Day3Dont -> mul_enabled := false; 0
    | Day3Mul (x, y) -> if !mul_enabled then x * y else 0 in
  parse_day3_program s
  >>| eval
  |> fold ~init:0 ~f:(+)

val eval_day3_program : string -> int = <fun>


In [45]:
eval_day3_program "xmul(2,4)&mul[3,7]!^don't()_mul(5,5)+mul(32,64](mul(11,8)undo()?mul(8,5))"

- : int = 48


In [46]:
In_channel.read_all "day3_input.txt" |> eval_day3_program

- : int = 82868252


## Day 4

### Part 1

Today we are doing word searches, which I guess means we need arrays.  Let's see how this works.

In [73]:
let read_wordsearch file_name =
  let lines = In_channel.read_lines file_name in
  let rows = length lines in
  let cols = String.length (nth_exn lines 0) in
  let grid = Array.make_matrix rows cols '.' in
  for i = 0 to rows-1 do
    for j = 0 to cols-1 do
      grid.(i).(j) <- String.get (nth_exn lines i) j
    done
  done;
  grid

val read_wordsearch : Base.string -> char Core.Array.t Core.Array.t = <fun>


In [74]:
read_wordsearch "day4_example.txt"

- : char Core.Array.t Core.Array.t =
[|[|'M'; 'M'; 'M'; 'S'; 'X'; 'X'; 'M'; 'A'; 'S'; 'M'|];
  [|'M'; 'S'; 'A'; 'M'; 'X'; 'M'; 'S'; 'M'; 'S'; 'A'|];
  [|'A'; 'M'; 'X'; 'S'; 'X'; 'M'; 'A'; 'A'; 'M'; 'M'|];
  [|'M'; 'S'; 'A'; 'M'; 'A'; 'S'; 'M'; 'S'; 'M'; 'X'|];
  [|'X'; 'M'; 'A'; 'S'; 'A'; 'M'; 'X'; 'A'; 'M'; 'M'|];
  [|'X'; 'X'; 'A'; 'M'; 'M'; 'X'; 'X'; 'A'; 'M'; 'A'|];
  [|'S'; 'M'; 'S'; 'M'; 'S'; 'A'; 'S'; 'X'; 'S'; 'S'|];
  [|'S'; 'A'; 'X'; 'A'; 'M'; 'A'; 'S'; 'A'; 'A'; 'A'|];
  [|'M'; 'A'; 'M'; 'M'; 'M'; 'X'; 'M'; 'M'; 'M'; 'M'|];
  [|'M'; 'X'; 'M'; 'X'; 'A'; 'X'; 'M'; 'A'; 'S'; 'X'|]|]


In [107]:
let count_matches word grid =
  let count_matches_at i j di dj =
    let len = String.length word in
    let found = ref 1 in
    for n = 0 to len-1 do
      try
        let word_char = String.get word n in
        let grid_char = grid.(i + di * n).(j + dj * n) in
          if Char.(<>) word_char grid_char then found := 0
      with
        Invalid_argument _ -> found := 0
    done; !found in
  let matches = ref 0 in
    let rows = Array.length grid in
    let cols = Array.length grid.(0) in
    let dirs = [(-1, -1); (0, -1); (1, -1);
                (-1,  0);          (1,  0);
                (-1,  1); (0,  1); (1,  1)] in
    for i = 0 to rows-1 do
      for j = 0 to cols-1 do
        let matches_here = dirs
          >>| (fun((di, dj)) -> count_matches_at i j di dj)
          |> fold ~init:0 ~f:(+) in
          matches := !matches + matches_here
      done
    done; !matches

val count_matches : string -> Core.Char.t Core.Array.t Core.Array.t -> int =
  <fun>


Why doesn't the inequality operator `<>` work right for characters?  That's weird.  This Jane Street library is banana pancakes.

In [108]:
read_wordsearch "day4_example.txt" |> count_matches "XMAS"

- : int = 18


In [109]:
read_wordsearch "day4_input.txt" |> count_matches "XMAS"

- : int = 2524


### Part 2

Oh good we have to search a different way.  Let's see...

In [125]:
let count_x_mas grid =
  let count_crossing_matches_at i j =
    try
      let ul = grid.(i-1).(j-1) in
      let ur = grid.(i-1).(j+1) in
      let dl = grid.(i+1).(j-1) in
      let dr = grid.(i+1).(j+1) in
      if Char.(grid.(i).(j) = 'A') &&
        (Char.(ul = 'M') && Char.(dr = 'S') ||
         Char.(ul = 'S') && Char.(dr = 'M')) &&
        (Char.(ur = 'M') && Char.(dl = 'S') ||
         Char.(ur = 'S') && Char.(dl = 'M')) then 1
      else 0
    with
      Invalid_argument _ -> 0 in
  let matches = ref 0 in
    let rows = Array.length grid in
    let cols = Array.length grid.(0) in
    for i = 0 to rows-1 do
      for j = 0 to cols-1 do
        let matches_here = count_crossing_matches_at i j in
          matches := !matches + matches_here
      done
    done; !matches

val count_x_mas : Core.Char.t Core.Array.t Core.Array.t -> int = <fun>


In [121]:
count_x_max

- : bool = false
